In [47]:
import os

import boto3
import hcl2

In [44]:
from helper import wait_until

In [1]:
TERRAFORM_FOLDER = '/terraform/eberron-agent'

In [57]:
with open(os.path.join(TERRAFORM_FOLDER, "provider.tf"), "r") as file:
    terraform = hcl2.load(file)
with open(os.path.join(TERRAFORM_FOLDER, "variables.tf"), "r") as file:
    terraform = terraform | hcl2.load(file)
# REGION = terraform['provider'][0]['aws']['region']
REGION = terraform['variable'][0]['region']['default']
CLUSTER_NAME = terraform['variable'][0]['cluster_name']['default']

In [52]:
session = boto3.Session(region_name=REGION)
eks_client = session.client('eks')

In [16]:
!terraform -chdir=$TERRAFORM_FOLDER init -upgrade

Initializing the backend...
Upgrading modules...
- eks in .terraform/modules/eks
- eks.eks_managed_node_group in .terraform/modules/eks/modules/eks-managed-node-group
- eks.eks_managed_node_group.user_data in .terraform/modules/eks/modules/_user_data
- eks.fargate_profile in .terraform/modules/eks/modules/fargate-profile
- eks.kms in .terraform/modules/eks.kms
- eks.self_managed_node_group in .terraform/modules/eks/modules/self-managed-node-group
- eks.self_managed_node_group.user_data in .terraform/modules/eks/modules/_user_data
- vpc in .terraform/modules/vpc
Initializing provider plugins...
- Finding hashicorp/time versions matching ">= 0.9.0"...
- Finding hashicorp/null versions matching ">= 3.0.0"...
- Finding hashicorp/kubernetes versions matching "~> 2.35"...
- Finding hashicorp/aws versions matching ">= 4.33.0, ~> 5.0, >= 5.46.0, >= 5.83.0"...
- Finding latest version of hashicorp/random...
- Finding hashicorp/tls versions matching ">= 3.0.0"...
- Finding hashicorp/cloudinit ve

In [38]:
!terraform -chdir=$TERRAFORM_FOLDER validate

Success! The configuration is valid.



In [40]:
!terraform -chdir=$TERRAFORM_FOLDER plan -out=./plan.out

data.aws_availability_zones.available: Reading...
module.eks.data.aws_caller_identity.current[0]: Reading...
module.eks.data.aws_iam_policy_document.assume_role_policy[0]: Reading...
module.eks.module.eks_managed_node_group["regular_nodes"].data.aws_caller_identity.current: Reading...
module.eks.module.eks_managed_node_group["gpu_nodes"].data.aws_partition.current: Reading...
module.eks.module.kms.data.aws_partition.current[0]: Reading...
module.eks.module.eks_managed_node_group["gpu_nodes"].data.aws_caller_identity.current: Reading...
module.eks.module.eks_managed_node_group["regular_nodes"].data.aws_partition.current: Reading...
module.eks.data.aws_partition.current[0]: Reading...
module.eks.module.kms.data.aws_partition.current[0]: Read complete after 0s [id=aws]
module.eks.data.aws_partition.current[0]: Read complete after 0s [id=aws]
module.eks.module.eks_managed_node_group["regular_nodes"].data.aws_partition.current: Read complete after 0s [id=aws]
module.eks.module.eks_managed_n

In [ ]:
!terraform -chdir=$TERRAFORM_FOLDER apply ./plan.out -auto-approve

module.eks.module.kms.data.aws_caller_identity.current[0]: Reading...
data.aws_availability_zones.available: Reading...
module.eks.module.eks_managed_node_group["gpu_nodes"].data.aws_caller_identity.current: Reading...
module.eks.module.eks_managed_node_group["regular_nodes"].data.aws_caller_identity.current: Reading...
module.eks.data.aws_partition.current[0]: Reading...
module.eks.data.aws_caller_identity.current[0]: Reading...
module.eks.data.aws_iam_policy_document.assume_role_policy[0]: Reading...
module.eks.data.aws_partition.current[0]: Read complete after 0s [id=aws]
module.eks.module.kms.data.aws_partition.current[0]: Reading...
module.eks.module.eks_managed_node_group["regular_nodes"].data.aws_partition.current: Reading...
module.eks.module.eks_managed_node_group["gpu_nodes"].data.aws_partition.current: Reading...
module.eks.module.kms.data.aws_partition.current[0]: Read complete after 0s [id=aws]
module.eks.module.eks_managed_node_group["regular_nodes"].data.aws_iam_policy_d

In [ ]:
wait_until(eks_client.describe_nodegroup, {'clusterName': CLUSTER_NAME, 'nodegroupName': node_group_name}, is_node_group_active)

In [60]:
response = eks_client.describe_nodegroup(clusterName=CLUSTER_NAME, nodegroupName='regular_nodes-20250122011909763300000019')
response

{'ResponseMetadata': {'RequestId': '9fb45436-bc85-4ae6-b7a5-400efd3a3b3e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 22 Jan 2025 01:58:55 GMT',
   'content-type': 'application/json',
   'content-length': '1781',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9fb45436-bc85-4ae6-b7a5-400efd3a3b3e',
   'access-control-allow-origin': '*',
   'access-control-allow-headers': '*,Authorization,Date,X-Amz-Date,X-Amz-Security-Token,X-Amz-Target,content-type,x-amz-content-sha256,x-amz-user-agent,x-amzn-platform-id,x-amzn-trace-id',
   'x-amz-apigw-id': 'ExGTAEEA4osEQtw=',
   'access-control-allow-methods': 'GET,HEAD,PUT,POST,DELETE,OPTIONS',
   'access-control-expose-headers': 'x-amzn-errortype,x-amzn-errormessage,x-amzn-trace-id,x-amzn-requestid,x-amz-apigw-id,date',
   'x-amzn-trace-id': 'Root=1-679050df-4534ae534fa720e66a5d52ab'},
  'RetryAttempts': 0},
 'nodegroup': {'nodegroupName': 'regular_nodes-20250122011909763300000019',
  'nodegroupArn': 'arn:aws:eks:ca-central-1

In [ ]:
# Your current IAM principal doesn’t have access to Kubernetes objects on this cluster.
# This may be due to the current user or role not having Kubernetes RBAC permissions to describe cluster resources or not having an entry in the cluster’s auth config map.

# NodeCreationFailure                          Instances failed to join the kubernetes cluster